In [2]:
!pip install transformers datasets peft torch accelerate pillow bitsandbytes # bitsandbytes often needed for efficient loading

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled n

In [3]:
import os
import json
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import Blip2ForConditionalGeneration, Blip2Processor, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType

In [4]:
class VQADataset(Dataset):
    def __init__(self, json_dirs, base_img_path, processor):
        self.samples = []
        self.processor = processor
        self.base_img_path = base_img_path

        print(f"📁 Scanning JSON folders: {json_dirs}")

        for dir_path in ([json_dirs] if isinstance(json_dirs, str) else json_dirs):
            for root, _, files in os.walk(dir_path):
                for file in files:
                    if file.endswith(".json"):
                        json_path = os.path.join(root, file)
                        with open(json_path, "r") as f:
                            data = json.load(f)

                        image_rel_path = data["image_path"].replace("\\", "/")

                        # Remove everything before and including 'abo-images-small/'
                        if "abo-images-small/" in image_rel_path:
                            image_rel_path = image_rel_path.split("abo-images-small/", 1)[1]

                        image_path = os.path.join(base_img_path, image_rel_path)

                        for qa in data["qa_pairs"]:
                            self.samples.append({
                                "image_path": image_path,
                                "question": qa["question"],
                                "answer": qa["answer"]
                            })

        print(f"✅ Loaded {len(self.samples)} samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample["image_path"]).convert("RGB")

        inputs = self.processor(
            images=image,
            text=sample["question"],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=128
        )

        labels = self.processor.tokenizer(
            sample["answer"],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=10
        ).input_ids

        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = labels.squeeze(0)

        # Remove inputs_embeds if it exists
        if "inputs_embeds" in inputs:
            del inputs["inputs_embeds"]

        return inputs

In [5]:
def compute_metrics(pred):
    preds = pred.predictions
    labels = pred.label_ids

    tokenizer = Blip2Processor.from_pretrained("Salesforce/blip-vqa-base").tokenizer
    pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    correct = sum(p.strip().lower() == l.strip().lower() for p, l in zip(pred_texts, label_texts))
    acc = correct / len(label_texts)
    print(f"✅ Accuracy: {acc:.4f}")
    return {"accuracy": acc}

In [6]:
from transformers import BlipProcessor, BlipForQuestionAnswering, TrainingArguments, Trainer
import torch
from torch.utils.data import random_split
from datasets import load_dataset

In [7]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b29414a023b5971c5fcbf64a5906ef529178ab63bd50c9400462f4947de4561c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [8]:
!pip install rouge-score nltk sacrebleu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.2 MB/s eta 0:00:00


In [9]:
pip install bert-score nltk rouge-score fuzzywuzzy python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [11]:
!git clone https://github.com/neulab/BARTScore.git

fatal: destination path 'BARTScore' already exists and is not an empty directory.


In [12]:
%cd BARTScore


/kaggle/working/BARTScore


In [13]:
!pip install transformers sentencepiece rouge-score


In [14]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [15]:
import sys
sys.path.append('/kaggle/working/BARTScore')

In [16]:
!pip install bert-score
!pip install scikit-learn

In [18]:
# import shutil
# shutil.rmtree("/kaggle/working/vqa_eval_output", ignore_errors=True)

In [19]:
import os
import json
import time
import numpy as np
import torch
from torch.amp import autocast # Corrected import for autocast
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import BlipProcessor, BlipForQuestionAnswering
from accelerate import Accelerator
from sklearn.metrics import f1_score
from bert_score import score as bert_score_fn
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import single_meteor_score
from fuzzywuzzy import fuzz
from bart_score import BARTScorer
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.translate.bleu_score import SmoothingFunction

# Ensure necessary NLTK data is downloaded
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt', quiet=True)


os.environ["TOKENIZERS_PARALLELISM"] = "false"

# === Accelerator and Device Setup ===
accelerator = Accelerator()
device = accelerator.device

# === Load Model and Processor ===
load_path = "/kaggle/input/blip-finetunedmodel-versions/model_latest_v1"
try:
    model = BlipForQuestionAnswering.from_pretrained(load_path)
    processor = BlipProcessor.from_pretrained(load_path)
except Exception as e:
    print(f"Could not load model/processor from {load_path}. Falling back to default BLIP model for demonstration.")
    # Fallback for local testing if the Kaggle path is not accessible
    model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
    processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

model = accelerator.prepare(model)

# === Initialize BARTScorer ===
bart_scorer = None
try:
    bart_scorer_device = 'cuda' if device.type == 'cuda' else 'cpu'
    bart_scorer = BARTScorer(device=bart_scorer_device, checkpoint='facebook/bart-large-cnn')
    print(f"BARTScorer initialized on {bart_scorer_device}.")
except Exception as e:
    print(f"Could not initialize BARTScorer: {e}. BARTScore will not be calculated.")
    bart_scorer = None



# === Resume Directory and File ===
resume_dir = "/kaggle/working/vqa_eval_output_base"
os.makedirs(resume_dir, exist_ok=True)
resume_file = os.path.join(resume_dir, "eval_resume.json")

# === Resume Logic ===
if os.path.exists(resume_file):
    try:
        with open(resume_file, "r") as f:
            resume_data = json.load(f)
        processed_indices = set(resume_data["indices"])
        predicted_all = resume_data["predicted"]
        true_all = resume_data["true"]
        correct = resume_data["correct"]
        total = resume_data["total"]
        print(f"Resume data loaded: {len(predicted_all)} predictions, {len(true_all)} true values.")
    except json.JSONDecodeError:
        print("Error decoding resume file. Starting evaluation from scratch.")
        processed_indices = set()
        predicted_all, true_all = [], []
        correct = total = 0
else:
    processed_indices = set()
    predicted_all, true_all = [], []
    correct = total = 0
    print("Default values initialized.")

# === Evaluation ===

# === Evaluation ===


# === Metrics Calculation ===
if total == 0:
    print("\nNo samples were processed. Cannot calculate metrics.")
    exit()
    
print("calculating em")
exact_match = 100 * correct / total
bert_score_f1 = bert_score_precision = bert_score_recall = 0.0
avg_bleu = avg_rouge_l = avg_meteor = 0.0
avg_jaccard = avg_dice = avg_lcs = avg_token_overlap = avg_fuzzy = 0.0
avg_vqa_acc = 0.0
avg_vccs = 0.0
bart_score_value = 0.0

if len(predicted_all) > 0 and len(true_all) > 0:
    print("calculating Bert Scores")
    P, R, F1 = bert_score_fn(predicted_all, true_all, lang="en", verbose=False)
    bert_score_f1 = F1.mean().item()
    bert_score_precision = P.mean().item()
    bert_score_recall = R.mean().item()

    print("calculating Bleu")
    # BLEU score: Using word_tokenize for more robust tokenization
    smoother = SmoothingFunction().method1  # Optional: to avoid 0 BLEU for short sentences

    bleu_scores = [
        sentence_bleu([word_tokenize(ref)], word_tokenize(hyp), smoothing_function=smoother)
        for ref, hyp in zip(true_all, predicted_all)
    ]

    avg_bleu = np.mean(bleu_scores)

    print("calculating Rouge")
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l_scores = [scorer.score(ref, hyp)["rougeL"].fmeasure for ref, hyp in zip(true_all, predicted_all)]
    avg_rouge_l = np.mean(rouge_l_scores)

    print("calculating meteor")
    # METEOR score: FIX applied here - requires pre-tokenized hypothesis and reference
    meteor_scores = [single_meteor_score(word_tokenize(ref), word_tokenize(hyp)) for ref, hyp in zip(true_all, predicted_all)]
    avg_meteor = np.mean(meteor_scores)

    def jaccard_sim(a, b):
        # Changed to word_tokenize for more robust tokenization
        set_a, set_b = set(word_tokenize(a)), set(word_tokenize(b))
        return len(set_a & set_b) / len(set_a | set_b) if set_a | set_b else 0.0
    print("calculating jacord")
    
    jaccard_scores = [jaccard_sim(a, b) for a, b in zip(true_all, predicted_all)]
    avg_jaccard = np.mean(jaccard_scores)

    def dice_sim(a, b):
        # Changed to word_tokenize for more robust tokenization
        set_a, set_b = set(word_tokenize(a)), set(word_tokenize(b))
        return 2 * len(set_a & set_b) / (len(set_a) + len(set_b)) if set_a or set_b else 0.0
        
    print("calculating dice")
    dice_scores = [dice_sim(a, b) for a, b in zip(true_all, predicted_all)]
    avg_dice = np.mean(dice_scores)

    def lcs(a_tokens, b_tokens): # Function now expects token lists
        dp = np.zeros((len(a_tokens)+1, len(b_tokens)+1))
        for i in range(len(a_tokens)):
            for j in range(len(b_tokens)):
                if a_tokens[i] == b_tokens[j]:
                    dp[i+1][j+1] = dp[i][j] + 1
                else:
                    dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
        return dp[len(a_tokens)][len(b_tokens)]

    print("calculating lcs")

    # Applying word_tokenize to `a` and `b` for word-level LCS calculation
    lcs_ratios = [lcs(word_tokenize(a), word_tokenize(b)) / max(len(word_tokenize(a)), len(word_tokenize(b))) if max(len(word_tokenize(a)), len(word_tokenize(b))) > 0 else 0
                  for a, b in zip(true_all, predicted_all)]
    avg_lcs = np.mean(lcs_ratios)

    def token_overlap(a, b):
        # Changed to word_tokenize for more robust tokenization
        a_tokens, b_tokens = word_tokenize(a), word_tokenize(b)
        return len(set(a_tokens) & set(b_tokens)) / len(set(a_tokens) | set(b_tokens)) if a_tokens or b_tokens else 0.0

    print("calculating token overlap")
    token_overlaps = [token_overlap(a, b) for a, b in zip(true_all, predicted_all)]
    avg_token_overlap = np.mean(token_overlaps)

    print("calculating fuzzy")
    fuzzy_scores = [fuzz.ratio(a, b)/100 for a, b in zip(true_all, predicted_all)]
    avg_fuzzy = np.mean(fuzzy_scores)

    print("calculating vqa")
    vqa_accs = [1.0 if p == t else 0.0 for p, t in zip(predicted_all, true_all)]
    avg_vqa_acc = np.mean(vqa_accs)

    # === Visual-Contextual Consistency Score (VCCS) ===
    def visual_contextual_consistency_score(ground_truth, predicted_answer):
        gt_tokens = word_tokenize(ground_truth.lower())
        pred_tokens = word_tokenize(predicted_answer.lower())

        gt_tokens = [word for word in gt_tokens if word not in string.punctuation]
        pred_tokens = [word for word in pred_tokens if word not in string.punctuation]

        set_gt = set(gt_tokens)
        set_pred = set(pred_tokens)

        if not set_gt and not set_pred:
            return 1.0
        if not set_gt or not set_pred:
            return 0.0

        intersection = len(set_gt.intersection(set_pred))
        union = len(set_gt.union(set_pred))

        return intersection / union
    print("calculating vccs")
    vccs_scores = []
    for gt, pred in zip(true_all, predicted_all):
        vccs_scores.append(visual_contextual_consistency_score(gt, pred))
    avg_vccs = np.mean(vccs_scores)

    # === BARTScore Calculation ===
    # === Calculate Score ===
    print("calculating bart")
    if bart_scorer is not None:
        try:
            # Make sure both predicted_all and true_all are lists of strings
            bart_scores_list = bart_scorer.score(predicted_all, true_all, batch_size=4)
            bart_score_value = np.mean(bart_scores_list)
        except Exception as e:
            print(f"Error calculating BARTScore: {e}")
            bart_score_value = 0.0
    else:
        print("BARTScorer not initialized, skipping BARTScore calculation.")


# === Print All Metrics ===
print(f"\n📊 Evaluation Metrics:")
print(f"✅ Exact Match (EM): {exact_match:.2f}%")
print(f"🤖 BERTScore - Precision: {bert_score_precision:.4f}")
print(f"🤖 BERTScore - Recall:    {bert_score_recall:.4f}")
print(f"🤖 BERTScore - F1:        {bert_score_f1:.4f}")
print(f"🔹 BLEU Score:            {avg_bleu:.4f}")
print(f"🔹 ROUGE-L:               {avg_rouge_l:.4f}")
print(f"🔹 METEOR:                {avg_meteor:.4f}")
print(f"🔹 Jaccard Similarity:    {avg_jaccard:.4f}")
print(f"🔹 Sørensen–Dice Coefficient: {avg_dice:.4f}")
print(f"🔹 LCS Ratio:             {avg_lcs:.4f}")
print(f"🔹 Token-Level Overlap:   {avg_token_overlap:.4f}")
print(f"🔹 Fuzzy Matching Score:  {avg_fuzzy:.4f}")
print(f"🔹 VQA Accuracy:          {avg_vqa_acc*100:.2f}%")
print(f"🔹 Visual-Contextual Consistency Score (VCCS): {avg_vccs:.4f}")
print(f"🔹 BARTScore:             {bart_score_value:.4f}")


# --- Snippet to save metrics to a .txt file ---

metrics_file_path = os.path.join(resume_dir, "evaluation_metrics.txt")

with open(metrics_file_path, "w") as f:
    f.write("📊 Evaluation Metrics:\n")
    f.write(f"✅ Exact Match (EM): {exact_match:.2f}%\n")
    f.write(f"🤖 BERTScore - Precision: {bert_score_precision:.4f}\n")
    f.write(f"🤖 BERTScore - Recall:    {bert_score_recall:.4f}\n")
    f.write(f"🤖 BERTScore - F1:        {bert_score_f1:.4f}\n")
    f.write(f"🔹 BLEU Score:            {avg_bleu:.4f}\n")
    f.write(f"🔹 ROUGE-L:               {avg_rouge_l:.4f}\n")
    f.write(f"🔹 METEOR:                {avg_meteor:.4f}\n")
    f.write(f"🔹 Jaccard Similarity:    {avg_jaccard:.4f}\n")
    f.write(f"🔹 Sørensen–Dice Coefficient: {avg_dice:.4f}\n")
    f.write(f"🔹 LCS Ratio:             {avg_lcs:.4f}\n")
    f.write(f"🔹 Token-Level Overlap:   {avg_token_overlap:.4f}\n")
    f.write(f"🔹 Fuzzy Matching Score:  {avg_fuzzy:.4f}\n")
    f.write(f"🔹 VQA Accuracy:          {avg_vqa_acc*100:.2f}%\n")
    f.write(f"🔹 Visual-Contextual Consistency Score (VCCS): {avg_vccs:.4f}\n")
    f.write(f"🔹 BARTScore:             {bart_score_value:.4f}\n")

print(f"\nAll metrics saved to: {metrics_file_path}")

Could not load model/processor from /kaggle/input/blip-finetunedmodel-versions/model_latest_v1. Falling back to default BLIP model for demonstration.


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating Bleu
calculating Rouge
calculating meteor
calculating jacord
calculating dice
calculating lcs
calculating token overlap
calculating fuzzy
calculating vqa
calculating vccs
calculating bart

📊 Evaluation Metrics:
✅ Exact Match (EM): 13.28%
🤖 BERTScore - Precision: 0.9487
🤖 BERTScore - Recall:    0.9310
🤖 BERTScore - F1:        0.9388
🔹 BLEU Score:            0.0248
🔹 ROUGE-L:               0.1418
🔹 METEOR:                0.0853
🔹 Jaccard Similarity:    0.1369
🔹 Sørensen–Dice Coefficient: 0.1387
🔹 LCS Ratio:             0.1370
🔹 Token-Level Overlap:   0.1369
🔹 Fuzzy Matching Score:  0.2818
🔹 VQA Accuracy:          13.28%
🔹 Visual-Contextual Consistency Score (VCCS): 0.1373
🔹 BARTScore:             -6.3153

All metrics saved to: /kaggle/working/vqa_eval_output_base/evaluation_metrics.txt
